# Analisis de la Varianza: Ejemplo de mejora de tratamiento

Se analiza la mejora tras usar tratamientos para tratar el acne. Hiru tratamendu ezberdin erabili ondoren, lorturiko hobekuntza portzentajeak aztertu nahi dugu (ver diapositiva 5 de la presentación del tema Analisis de la varianza). Suponemos que el porcentaje de mejora es una variable aleatoria normal. ¿Hay diferencia significativa entre los tres tratamientos a un nivel de significación del $5\%$?


In [3]:
# leer datos 
library("readxl")
Datos <-read_excel("AOV_Tratamiento.xls")
str(Datos)

# Pasamos la variable tratamiento a factor y la variable mejora a numerica
Datos$Tratamiento.f <-factor(Datos$Tratamiento)
Datos$Mejora <-as.numeric(Datos$Mejora)
#str(Hobekuntza)
#Hobekuntza

tibble [35 × 2] (S3: tbl_df/tbl/data.frame)
 $ Mejora     : chr [1:35] "48.6" "49.4" "50.1" "49.8" ...
 $ Tratamiento: chr [1:35] "I" "I" "I" "I" ...


Comenzamos por presentar cosas vistas en estadística descriptiva

In [5]:
#install.packages("RcmdrMisc")
library("RcmdrMisc")
numSummary(Datos[,"Mejora"], groups=Datos$Tratamiento.f,    
           statistics = c("mean", "sd", "quantiles"), quantiles=c(0,.25,.5,.75,1))

        mean       sd   0%   25%  50%    75% 100% Mejora:n
I   49.46000 1.732179 46.7 48.70 49.6 50.475 52.5       10
II  68.73333 2.000606 64.5 67.95 68.6 69.950 71.9       12
III 63.60000 2.222236 60.5 62.30 63.2 64.800 67.5       13

En el enunciado se afirma la normalidad de los datos por lo que no lo verificamos.

### Igualdad de varianzas

In [6]:
leveneTest(Datos$Mejora, Datos$Tratamiento.f, center=mean) 

,Df,F value,Pr(>F)
,<int>,<dbl>,<dbl>
group,2,0.4913992,0.6163106
,32,NA,NA



El test de Levene realiza un contraste en el que $H_0$ es que las varianzas son iguales. En este caso, el valor del estadístico es 0.4913992, los grados de libertad 2 y el p-valor 0.6163106. Por consiguiente, a un nivel de significación del $5\%$, no se rechaza la igualdad de varianzas.

Por tanto, todo indica que estamos en las condiciones para realizar el Analisis de la Varianza. Hemos de realizar el siguiente contraste de hipótesis: 


$H_0:\ \mu_I=\mu_{II}=\mu_{III}$

$H_1: \exists i,j$ non $\mu_j\neq \mu_i$



In [7]:
Anova2 <-  aov(Mejora ~ Tratamiento.f, data=Datos)
summary(Anova2)

              Df Sum Sq Mean Sq F value Pr(>F)    
Tratamiento.f  2 2133.7  1066.8     262 <2e-16 ***
Residuals     32  130.3     4.1                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Observamos que el p-valor es mucho más pequeño que 0.0001, por tanto concluimos a un nivel de significación del $5\%$ que hay diferencia en la eficacia media de los tratamientos contra el acna.


Ahora la pregunta es... ¿Todos los tratamientos son iguales? Para responder a esta pregunta usaremos las comparaciones multiples de medias. Hay distintos métodos, pero el que más se usa es el de Scheffé  eta Tukey. El de Tukey es el que da los resultados más fiables, pero requiere que los tamaños muestrales sean iguales. El metodo de Scheffé se puede usar en cualquier caso (por tanto es el más usado) pero es más conservador que el de Tukey, es decir, da p-valores más altos.

In [10]:
# vamos a ver las medias y quasidesviaciones estandar de la muestra. ¿Cuales parecen que son distintas?
numSummary(Datos$Mejora , groups=Datos$Tratamiento,    statistics=c("mean", "sd")) 

        mean       sd data:n
I   49.46000 1.732179     10
II  68.73333 2.000606     12
III 63.60000 2.222236     13

In [11]:
library(multcomp)
# aurreko paketea kargatu behar da glht komandoa erabiltzeko
multiplesTukey <- glht(Anova2, linfct = mcp(Tratamiento.f = "Tukey"))
summary(multiplesTukey)
confint(multiplesTukey)


	 Simultaneous Tests for General Linear Hypotheses

Multiple Comparisons of Means: Tukey Contrasts


Fit: aov(formula = Mejora ~ Tratamiento.f, data = Datos)

Linear Hypotheses:
              Estimate Std. Error t value Pr(>|t|)    
II - I == 0    19.2733     0.8640  22.308   <1e-05 ***
III - I == 0   14.1400     0.8487  16.660   <1e-05 ***
III - II == 0  -5.1333     0.8078  -6.355   <1e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
(Adjusted p values reported -- single-step method)



	 Simultaneous Confidence Intervals

Multiple Comparisons of Means: Tukey Contrasts


Fit: aov(formula = Mejora ~ Tratamiento.f, data = Datos)

Quantile = 2.4557
95% family-wise confidence level
 

Linear Hypotheses:
              Estimate lwr     upr    
II - I == 0   19.2733  17.1517 21.3950
III - I == 0  14.1400  12.0558 16.2242
III - II == 0 -5.1333  -7.1170 -3.1497


Observamos los p-valores y nos damos cuenta que todas las comparaciones dos a dos tienen diferencias significativas. Por tanto, con un nivel de significación del $5\%$, la eficacia de todos tratamientos es distinta. ¿Pero cual es la mejor? ¿Y la peor?

Si miramos los intervalos de confianza, concluimos lo siguiente: Como $$I^{0.95}_{\mu_{II}-\mu_I}=(17.1517,21.395)$$ concluimos que $\mu_2>\mu_1$ con un nivel de significación del $5\%$. Además, vemos que $$I^{0.95}_{\mu_{III}-\mu_I}=(12.0558,16.2242),$$ concluimos con un  nivel de significación del $5\%$ que  $\mu_3 > \mu_1$ dela. Por ultimo, como $$I^{0.95}_{\mu_{III}-\mu_{II}}=(-7.117,-3.1497),$$ concluimos con un nivel de significación del $5\%$ que $\mu_3 < \mu_2$. Por tanto, se cumple que $\mu_2 > \mu_3 > \mu_1$. Es decir, el Tratamiento 2 es el de mayor eficacia y el tratamiento 1 el de peor.